In [11]:
from flask import Flask, render_template, jsonify, make_response
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pmdarima as pm
from sklearn.metrics import mean_squared_error
import io
import base64 

In [12]:
dates= pd.read_csv("../data/weather_data.csv")

In [13]:
app = Flask(__name__)

# Sample weather data generation

dates = pd.date_range(start='2018-01-01', periods=72, freq='M')
avgtempC = np.random.uniform(low=0, high=30, size=len(dates))
weather_df = pd.DataFrame(data={'date': dates, 'avgtempC': avgtempC})

# Preprocess the data
weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df.set_index('date', inplace=True)

# Train and test data preparation
train_df = weather_df['2018':'2022'].resample('M').mean()
test_df = weather_df['2023':'2024'].resample('M').mean()

@app.route('/forecast_temperature')
def forecast_temperature():
    # Step 1: Train the model on training data
    model = pm.auto_arima(train_df, seasonal=True, m=12)
    model.fit(train_df)

    # Step 2: Predict future temperatures
    forecast = model.predict(n_periods=len(test_df))
    forecast_df = pd.DataFrame(forecast, index=test_df.index, columns=['Prediction'])

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(test_df, forecast_df))

    # Visualize the results
    plt.figure(figsize=(10, 5))
    plt.plot(train_df, label='Training Data')
    plt.plot(test_df, label='Actual Data', color='orange')
    plt.plot(forecast_df, label='Forecasted Data', color='red')
    plt.legend()
    plt.title('ARIMA Model Evaluation')
    plt.xlabel('Date')
    plt.ylabel('Temperature')

    # Save plot to BytesIO object
    img = io.BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    plot_url = base64.b64encode(img.getvalue()).decode()

    return render_template('index.html', rmse=rmse, plot_url=plot_url)
@app.route('/')
def index():
    return 'Witaj na stronie głównej!'

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)




 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use